In [ ]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [ ]:
%store -r tarjetas
%store -r bancos

In [ ]:
tarjeta_mastercard = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Mastercard'] #Filtro por la Tarjeta Mastercard
tarjeta_mastercard = tarjeta_mastercard.loc[:, (tarjeta_mastercard!= 0).any(axis=0)]
tarjeta_mastercard = tarjeta_mastercard.loc[tarjetas['Cod Comercio']== 4547438] #Hay que filtrar por la tarjeta Mastercad y por este numero de comercio

In [ ]:
tarjeta_mastercard["PERCEPCION IVA R.G. 2408  1,50/3,00 %"] = tarjeta_mastercard[
                  ['PERCEPCION IVA R.G. 2408   1,50 %', 'PERCEPCION IVA R.G. 2408   3,00 %']].sum(axis=1, min_count=1)
tarjeta_mastercard['VENTAS'] = tarjeta_mastercard['VENTAS C/DESCUENTO CONTADO'] + tarjeta_mastercard['VENTAS C/DTO CUOTAS FINANC. OTORG.']
tarjeta_mastercard['RETENCION ING.BRUTOS SIRTAC'] = (tarjeta_mastercard['RETENCION ING.BRUTOS SIRTAC']
                                                  + tarjeta_mastercard['AJUSTE SIRTAC OCT. 1RA QUIN']
                                                  + tarjeta_mastercard['QR  RETENCION IIBB BS. AS.'])  
tarjeta_mastercard['RETENCION IVA'] = tarjeta_mastercard['RETENCION IVA'] + tarjeta_mastercard['QR  RETENCION IVA']
tarjeta_mastercard['RETENCION IMP.GANANCIAS'] = tarjeta_mastercard['RETENCION IMP.GANANCIAS'] + tarjeta_mastercard['QR  RETENCION GANANCIAS']

tarjeta_mastercard["TOTAL DEDUCCIONES"] = tarjeta_mastercard[["TOTAL DEDUCCIONES", "TOTAL PAGOS DE COMERCIOS",'CARGOS DE ENTIDAD']
                                                             ].sum(axis=1, min_count=1)
tarjeta_mastercard['IVA 21%'] = tarjeta_mastercard[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%','IVA PROMO CUOTAS AHORA 12/18 - 21,00%'
                              , 'IVA RI SIST CUOTAS','IVA RI CRED.FISC.COMERCIO S/DTO F.OTORG'
                              ,'IVA RI. CARGO LIQUID. ELECT.', 'IVA RI SERV.OPER. INT.']].sum(axis=1, min_count=1)
tarjeta_mastercard['ARANCEL'] = (tarjeta_mastercard['ARANCEL'] + tarjeta_mastercard['CARGO  SISTEMA CUOTAS MENS'] +
                                tarjeta_mastercard['CARGO LIQUIDACION ELECTR.']+ tarjeta_mastercard['SERVICIO OPER. INTERNAC.'])
tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = (tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS']
                                                                              + tarjeta_mastercard['PROMO CUOTAS AHORA 12/18'])
tarjeta_mastercard['Iva 10,5%'] = (tarjeta_mastercard['IVA PROMO CUOTAS AHORA 12/18 - 10,50%'] 
                               + tarjeta_mastercard['IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50'])
tarjeta_mastercard = tarjeta_mastercard.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS', 'ARANCEL', 'IVA 21%', 'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18',
       'Iva 10,5%', 'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC',"PERCEPCION IVA R.G. 2408  1,50/3,00 %",'REEMBOLSOS DE ENTIDAD' ,
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION']) #reordeno columnas, aca ya quedan en el orden que es necesario
tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_mastercard['Iva 10,5%']*100/10.5
tarjeta_mastercard['ARANCEL'] = tarjeta_mastercard['IVA 21%']*100/21            

tarjeta_mastercard.insert(7, 'Iva 21 control', tarjeta_mastercard['IVA 21%']/tarjeta_mastercard['ARANCEL'])
                                #agrego columna de control para IVA 21%
tarjeta_mastercard.insert(10, 'Iva 10,5 control', 
                                tarjeta_mastercard['Iva 10,5%']/tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'])
                                #agrego columna de control para IVA 10,5%                                
tarjeta_mastercard = tarjeta_mastercard.assign(Diferencia = tarjeta_mastercard.iloc[:, 5:7].sum(axis=True) 
                         + tarjeta_mastercard.iloc[:, 8:10].sum(axis=True)
                         + tarjeta_mastercard.iloc[:, 11:18].sum(axis=True)
                         - tarjeta_mastercard['TOTAL DEDUCCIONES']) #agrego columna de control final
tarjeta_mastercard = tarjeta_mastercard.round(2)

In [ ]:
sum_columns = [
       'Nro Liquidacion', 'VENTAS', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'Iva 10,5%',
       'Iva 10,5 control', 'RETENCION IVA', 'PER B.A.I.BR.DN.01/04',
       'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC', "PERCEPCION IVA R.G. 2408  1,50/3,00 %",
       'REEMBOLSOS DE ENTIDAD', 'TOTAL DEDUCCIONES',
       'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_mastercard=tarjeta_mastercard.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [ ]:
tarjeta_mastercard= tarjeta_mastercard.drop(['Diferencia', 'Iva 21 control', 'Iva 10,5 control'], axis=1)

In [ ]:
writer = ExcelWriter('/content/drive/MyDrive/Colab Notebooks/Le Utthe/Tarjetas/10-22/Mastercard1022-2.xlsx')
tarjeta_mastercard.to_excel(writer, 'Hoja de datos', index=True)
writer.save()